Step 0: Importing Relevant Packages (0%)

In [1]:
import math

# Web Scraping Libraries
from bs4 import BeautifulSoup  # for scraping
import requests  # for sending HTTP requests
# Data Processing Libraries
import csv
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Step 1: Crawl a Real World Data Set (2%)

In [169]:
url = "https://www.worldometers.info/coronavirus/#main_table"  # Web address of data
response = requests.get(url)  # Performing a get request for the data at the url location
soup_object = BeautifulSoup(response.content)  # Parsing the request response to create a soup_object that can be searched for a table
covid_table = soup_object.find("table")  # Searching the soup_object for a table

with open("Live_Covid_Data.csv", "w", newline="") as csv_file:  # Opening a CSV file to be written into
    writer = csv.writer(csv_file)  # Defining the writer as a csv writer that writes to the opened file

    for row in covid_table.find_all("tr"):  # Iterate through the rows of the table, "tr" points to a row in the table
        cells = row.find_all("td")  # Get the cells in the row, "td" points to a cell in the row

        if cells:  # If there are cells...
            values = [cell.text for cell in cells]  # Values is a list of values for each cell in the row of cells
            writer.writerow(values)  # Write the values to the CSV


Step 2: Data Preparation and Cleaning (5%)

In [3]:
# Creating list of column names
col_names = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered", "Active Cases", "Critical Cases",
             "Total Cases / 1M People", "Total Deaths / 1M People", "Total Tests", "Total Tests / 1M People", "Population", "Continent", "One Case / n People",
             "One Death / n People", "One Test / n People", "New Cases / 1M People", "New Deaths / 1M People", "Active Cases / 1M People"]

# Reading the csv to a dataframe with the column names specified, encoding with "latin-1" to get around "utf-8" decoding issue.
df = pd.read_csv("Live_Covid_Data.csv", names=col_names, header=None, encoding="latin-1")

# Tidying up the names of the continent by removing the "\n"'s on it, i.e. "\nNorth America\n"
old_continent_list = df.iloc[0:7]["Country"].tolist()  # Locating the continent names and putting them into a list
new_continent_list = []  # Defining a list for the formatted continent names

for old_continent in old_continent_list:  # Iterate through current continent names
     new_continent = "Antarctica" if old_continent == "\n\n" else old_continent.replace("\n", "")  # Removing \n from continent names/identifying Antarctica
     if new_continent == "Oceania":  # In "Continent" column countries are assigned Australia/Oceania not just Oceania
         new_continent = "Australia/Oceania"  # Renaming continent
     new_continent_list.append(new_continent)  # Addding the renamed continenet to the new list

# Changing the index to the name of the Country/Continent
df.set_index("Country", inplace=True)  # inplace=True updates the original datafram immediately

# Changing the index name to the re-formatted continents. set_value has been depricated, also before setting the index as Country the continent rows had
# an index of NaN so at[row, col] couldn't be used to assign values either. Therefore, application of the reformatted names had to after set_index was done.
n = 0
for new_continent in new_continent_list:  # Iterate through the new continent names
    df.rename(index={old_continent_list[n]: new_continent}, inplace=True)  # Replace the old continent name with corresponding new continent name
    n += 1

# Moving country continent and population to first/second column as it is important for comparison to total cases etc
country_pop_col = df.pop("Continent")
df.insert(0, "Continent", country_pop_col)
country_pop_col = df.pop("Population")  # Removes the column from DF while saving it to a variable
df.insert(1, "Population", country_pop_col)  # Replacing the column in first position using previously assigned variable

# Removing non-ccountries/non-territories from the DF
drop_list = ["Diamond Princess", "MS Zaandam", "Total:"]  # List of unwanted non-countries/non-territories
for item in drop_list:  # Iterate through list
    df.drop(index=item, axis=0, inplace=True)  # Permanently delete the corresponding row in place

# Calculating the population of each continent and the world
global_population = 0  # Defining global population as 0
for continent in new_continent_list:  # iterate through the continents
    # If a row has continent "continent" and is not the row for the continent itself, population is added to the corresponding list
    population_list = df.loc[(df["Continent"] == continent) & (df.index != continent), "Population"].tolist()
    continent_population = 0  # Defining the continent's population as 0

    for population in population_list:  # Iterate through all populations in the list
        new_population = re.sub(",", "", population)  # Removing the commas from the strings seperating 1000's
        continent_population += int(new_population)  # Adding the population to the continent populatuon

    global_population += continent_population  # Adding the continent population to the global population
    df.at[continent, "Population"] = continent_population  # Replacing the continents NaN population with the calculated population
df.at["World", "Population"] = global_population  # Replacing the world's NaN population with the calculated population

# Dropping Antarctica as its data is not useful
df.drop(index="Antarctica", axis=0, inplace=True)

# Handling missing values
df = df.replace(r"^\s*$", np.nan, regex=True)

# Remove commas from all columns and convert all columns to floats while avoiding non-numeric data
df = df.replace(",", "", regex=True)  # Removing commas from entire dataframe, regex=True automates searching and replacing
for column in df:  # Iterate through all columns in the dataframe
    try:  # Try the following experssion:
        df[column] = df[column].astype(float)  # Convert the column to floats
    except ValueError:  # If this error is caused, raise an exception for the Nan/string value
        pass  # Skip the conversion if the column contains non-numeric data

# Dropping columns with the format "New..." as "New" is ambiguous and not described on the website, they also contain a lot of NaN.
for column in df:
    if "New" in column:
        df.drop(columns=column, inplace=True)

# Calculating the total tests taken in each continent
global_total_tests = 0  # Defining the global total as 0
for continent in new_continent_list[:-1]:  # Iterating through continents, except antarctica
    continent_total_tests = 0  # Defining the continent total as 0
    test_list = df.loc[(df["Continent"] == continent) & (df.index != continent), "Total Tests"].tolist()  # Adding all test numbers to a list
    clean_test_list = [x for x in test_list if (math.isnan(x) == False)]  # Removing nan values from the list
    continent_total_tests = sum(clean_test_list)  # Summing all the values in the list
    global_total_tests += continent_total_tests  # Adding the continent total to the global total
    df.at[continent, "Total Tests"] = continent_total_tests  # Reassigning the continent total tests value to new calculated value
df.at["World", "Total Tests"] = global_total_tests  # Reassign global test total after for loop has completed

# Recalculating X/1M based on the live data from other columns where x = Total Cases, Total Deaths, Total Tests, Active Cases
for column in df:  # Iterate through columns in the DF
    if "1M People" in column:  # Checking if the column is a target for recalculation
        reduced_column = column.replace(" / 1M People", "")  # Formulating the column from which x/1M column will be calculated
        df[column] = (df[reduced_column] / df["Population"]) * 1000000  # Recalculating column

# Recalculating how many people there where per one case/death/test
df["One Case / n People"] = df["Population"] / df["Total Cases"]
df["One Death / n People"] = df["Population"] / df["Total Deaths"]
df["One Test / n People"] = df["Population"] / df["Total Tests"]

df

,Continent,Population,Total Cases,Total Deaths,Total Recovered,Active Cases,Critical Cases,Total Cases / 1M People,Total Deaths / 1M People,Total Tests,Total Tests / 1M People,One Case / n People,One Death / n People,One Test / n People,Active Cases / 1M People
Country,,,,,,,,,,,,,,,
North America,North America,5.981409e+08,121337450.0,1578727.0,116142918.0,3615805.0,9668.0,202857.632297,2639.389745,1.287663e+09,2.152775e+06,4.929566,378.875459,0.464517,6045.072162
Asia,Asia,4.711357e+09,206878311.0,1511747.0,193434481.0,11932083.0,11669.0,43910.559214,320.872961,2.348238e+09,4.984208e+05,22.773566,3116.498186,2.006337,2532.621398
Europe,Europe,7.475438e+08,242793984.0,1980561.0,236914861.0,3898562.0,7553.0,324788.958162,2649.424558,2.806916e+09,3.754851e+06,3.078922,377.440451,0.266322,5215.161716
South America,South America,4.376944e+08,66824087.0,1341748.0,64282682.0,1199657.0,10341.0,152672.916160,3065.490141,2.409430e+08,5.504821e+05,6.549950,326.212108,1.816589,2740.854994
Australia/Oceania,Australia/Oceania,4.346903e+07,13626060.0,23363.0,13343309.0,259388.0,176.0,313465.931952,537.463109,8.844942e+07,2.034769e+06,3.190139,1860.592818,0.491456,5967.190894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,Australia/Oceania,1.098200e+04,761.0,7.0,438.0,316.0,NaN,69295.210344,637.406665,2.050800e+04,1.867419e+06,14.431012,1568.857143,0.535498,28774.358040
Niue,Australia/Oceania,1.622000e+03,483.0,NaN,320.0,163.0,NaN,297780.517879,NaN,NaN,NaN,3.358178,NaN,NaN,100493.218249
Vatican City,Europe,7.990000e+02,29.0,NaN,29.0,0.0,NaN,36295.369212,NaN,NaN,NaN,27.551724,NaN,NaN,0.000000


Step 3: Exploratory Data Analysis (8%)

In [4]:
# Mean, sum, range, and
df.describe()

,Population,Total Cases,Total Deaths,Total Recovered,Active Cases,Critical Cases,Total Cases / 1M People,Total Deaths / 1M People,Total Tests,Total Tests / 1M People,One Case / n People,One Death / n People,One Test / n People,Active Cases / 1M People
count,2.350000e+02,2.350000e+02,2.300000e+02,2.150000e+02,2.160000e+02,148.000000,235.000000,230.000000,2.200000e+02,2.200000e+02,235.000000,230.000000,220.000000,216.000000
mean,1.014247e+08,8.479152e+06,8.731943e+04,8.788407e+06,2.893376e+05,815.878378,190215.787408,1233.812130,9.384579e+07,2.089764e+06,396.578047,14206.648691,10.527778,13390.698377
std,6.224031e+08,4.924169e+07,4.957105e+05,4.927483e+07,1.782182e+06,3740.887677,190419.006075,1263.628090,5.392495e+08,3.548503e+06,4095.713330,56403.262602,26.379282,46870.426227
min,7.990000e+02,1.000000e+01,1.000000e+00,2.000000e+00,0.000000e+00,1.000000,15.970334,1.597033,7.850000e+03,5.090634e+03,1.420538,154.388655,0.044340,0.000000
25%,5.822545e+05,2.876600e+04,2.307500e+02,2.499300e+04,1.335000e+02,5.000000,17987.625161,175.870168,3.639415e+05,1.891060e+05,3.239892,496.565488,0.421074,55.722758
50%,6.684849e+06,2.309700e+05,2.738000e+03,2.816190e+05,1.940000e+03,22.000000,123974.876715,794.034948,2.524787e+06,8.758852e+05,8.066150,1259.465141,1.141811,590.133026
75%,2.782692e+07,1.631720e+06,1.699600e+04,1.912317e+06,2.005850e+04,115.250000,308674.005366,2013.981620,1.699053e+07,2.375439e+06,55.709022,5686.160167,5.288352,3377.160933
max,7.944934e+09,6.642007e+08,6.694500e+06,6.361570e+08,2.134926e+07,40250.000000,703958.726870,6477.159861,6.882025e+09,2.255304e+07,62616.100000,626161.000000,196.439191,340116.235861


Step 4a: Question 1 (10%)

Step 4b: Question 2 (10%)

Step 4c: Question 3 (10%)

Step 5: Summary and Conclusion (5%)

Write summary/conc in here